# Search Evaluation 1 - Text Retrieval

Reference [LLM Zoomcamp 3.3.3 - Evaluation of Text Retrieval Techniques for RAG out
](https://www.youtube.com/watch?v=fdIV4xCsp0c)

# Step 0: Start up docker for ElasticSearch
```
docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

# Step 1: Load Documents into ElasticSearch

In [ ]:
import json
import numpy as np
np.float_ = np.float64
from tqdm.auto import tqdm
import pandas as pd
from elasticsearch import Elasticsearch

# download documents
!wget -nc https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/03-vector-search/eval/documents-with-ids.json

# load documents.json and flatten them
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

es_client = Elasticsearch('http://localhost:9200')

# create mapping
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"}
        }
    }
}

# create index
index_name = "course-questions"

es_client.indices.delete(index=index_name,ignore_unavailable=True)
es_client.indices.create(index=index_name,body=index_settings)

# add documents to index
for doc in tqdm(documents):
    try: 
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 2: Evaluate text search

In [ ]:
# define evaluation functions

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q['question'], q['course'])
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# define search function
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [ ]:
# load ground truth dataset
fn = 'https://raw.githubusercontent.com/slavaheroes/llm-zoomcamp/refs/heads/homeworks/03-vector-search/eval/ground-truth-data.csv'
ground_truth = pd.read_csv(fn).to_dict(orient='records')

In [ ]:
evaluate(ground_truth, elastic_search)